<a href="https://colab.research.google.com/github/Vivek-pareek/SMAI_Assignments/blob/main/DecisionTreesAndRandomForests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Decision Trees**

The Wisconsin Breast Cancer Dataset(WBCD) can be found here(https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data)

This dataset describes the characteristics of the cell nuclei of various patients with and without breast cancer. The task is to classify a decision tree to predict if a patient has a benign or a malignant tumour based on these features.

Attribute Information:
```
#  Attribute                     Domain
   -- -----------------------------------------
   1. Sample code number            id number
   2. Clump Thickness               1 - 10
   3. Uniformity of Cell Size       1 - 10
   4. Uniformity of Cell Shape      1 - 10
   5. Marginal Adhesion             1 - 10
   6. Single Epithelial Cell Size   1 - 10
   7. Bare Nuclei                   1 - 10
   8. Bland Chromatin               1 - 10
   9. Normal Nucleoli               1 - 10
  10. Mitoses                       1 - 10
  11. Class:                        (2 for benign, 4 for malignant)
```

In [ ]:
import pandas as pd
headers = ["ID","CT","UCSize","UCShape","MA","SECSize","BN","BC","NN",
           "Mitoses","Diagnosis"]
data = pd.read_csv('breast-cancer-wisconsin.data', na_values='?',    
         header=None, index_col=['ID'], names = headers) 
data = data.reset_index(drop=True)
data = data.fillna(0)
data.describe()

,CT,UCSize,UCShape,MA,SECSize,BN,BC,NN,Mitoses,Diagnosis
count,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000
mean,4.417740,3.134478,3.207439,2.806867,3.216023,3.463519,3.437768,2.866953,1.589413,2.689557
std,2.815741,3.051459,2.971913,2.855379,2.214300,3.640708,2.438364,3.053634,1.715078,0.951273
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,2.000000
25%,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000
50%,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000,2.000000
75%,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,5.000000,4.000000,1.000000,4.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


1. a) Implement a decision tree (you can use decision tree implementation from existing libraries).

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
import random
import numpy as np
import math

1. b) Train a decision tree object of the above class on the WBC dataset using misclassification rate, entropy and Gini as the splitting metrics.

In [ ]:
#We define decision trees using gini and entropy based methods
decision_tree_entropy = DecisionTreeClassifier(criterion = 'entropy')
decision_tree_gini = DecisionTreeClassifier(criterion = 'gini')

#get the feature columns in the data
feature_cols = headers[1 : -1]

#get the output classes
class_label = headers[-1]

#use library function to split data into test and train set
X_train, X_test, y_train, y_test = train_test_split(
    data[feature_cols], data[class_label], test_size = 0.2, random_state = 1)

#fit the models to our training data

decision_tree_entropy = decision_tree_entropy.fit(X_train, y_train)
y_pred_entropy = decision_tree_entropy.predict(X_test)

decision_tree_gini = decision_tree_gini.fit(X_train, y_train)
y_pred_gini = decision_tree_gini.predict(X_test)


1. c) Report the accuracies in each of the above splitting metrics and give the best result. 

In [ ]:
print('Accuracy using gini index for splitting is {:.2f}'.format(
    metrics.accuracy_score(y_test, y_pred_gini)))

print('Accuracy using entropy for splitting is {:.2f}'.format(
    metrics.accuracy_score(y_test, y_pred_entropy)))

Accuracy using gini index for splitting is 0.92
Accuracy using entropy for splitting is 0.94


1. d) Experiment with different approaches to decide when to terminate the tree (number of layers, purity measure, etc). Report and give explanations for all approaches. 

In [ ]:
'''
In the following code we will try the models with pruning the depth from 
(max_depth - 1) upto 2. We also use only sqrt of the number of features
at each split to determine the split
'''

# Method to prune decision trees at various depths and check accuracy score
#By default uses sqrt of features at each split
def experiment_depths(X_train, y_train, X_test, y_test, max_depth = None,
                      criterion = 'gini', max_features = "sqrt"):
  start_depth = 2

  max_score, max_score_depth = 0, 0

  for depth in range(start_depth, max_depth):
    emperical_decision_tree = DecisionTreeClassifier(max_depth = depth, 
                                                     criterion = criterion,
                                                     max_features = max_features)
    emperical_decision_tree = emperical_decision_tree.fit(X_train, y_train)
    y_pred = emperical_decision_tree.predict(X_test)
    score = metrics.accuracy_score(y_test, y_pred)
    if score > max_score:
      max_score = score
      max_score_depth = depth
    print("\naccuracy score for decision tree using {} at max depth {} is {:.4f}".
          format(criterion,
                 emperical_decision_tree.tree_.max_depth,
                 metrics.accuracy_score(y_test, y_pred)))
    
  return max_score, max_score_depth, criterion

entropy_max_score, entropy_max_depth, criterion_1 = experiment_depths(X_train, 
                  y_train, X_test, y_test, max_depth = 
                  (decision_tree_entropy.tree_.max_depth), criterion = 
                  'entropy')

gini_max_score, gini_max_depth, criterion_2 = experiment_depths(
    X_train, y_train, X_test, y_test, max_depth = (
        decision_tree_gini.tree_.max_depth), criterion = 'gini')

if gini_max_score > entropy_max_score :
  max_score = gini_max_score 
  print("\n\n The maximum accuracy in decision trees is {:.3f} after pruning at depth {} for criterion {}".format(gini_max_score, gini_max_depth,
                                         "gini"))
else :
  max_score = entropy_max_score 
  print("\n\n The maximum accuracy in decision trees is {:.3f} after pruning at depth {} for criterion {}".format(entropy_max_score, entropy_max_depth,"entropy"))  




accuracy score for decision tree using entropy at max depth 2 is 0.9214

accuracy score for decision tree using entropy at max depth 3 is 0.9000

accuracy score for decision tree using entropy at max depth 4 is 0.9643

accuracy score for decision tree using entropy at max depth 5 is 0.9500

accuracy score for decision tree using entropy at max depth 6 is 0.9357

accuracy score for decision tree using entropy at max depth 7 is 0.9571

accuracy score for decision tree using entropy at max depth 8 is 0.9357

accuracy score for decision tree using gini at max depth 2 is 0.9643

accuracy score for decision tree using gini at max depth 3 is 0.9429

accuracy score for decision tree using gini at max depth 4 is 0.9429

accuracy score for decision tree using gini at max depth 5 is 0.9571

accuracy score for decision tree using gini at max depth 6 is 0.9500

accuracy score for decision tree using gini at max depth 7 is 0.9143

accuracy score for decision tree using gini at max depth 8 is 0.9357

<u>explanation:</u>
<br>
<b>1.We perform depth pruning of the tree to avoid overfitting of the model to data, we can see from above output that the maximum accuracy occurs if we prune the tree earlier than the maximum depth.
<br>
<b>2.We use sqrt of features at each split. 
<br>
<b>Both these methods reduce overfitting of our model to data.

In [ ]:
class RandomForestBinaryClassifier():  
  '''
    Implementation of a simple random forest for binary classification.
    Here we create a forest of decision trees with sample size equal
    to the entire data size but each sample has data rows picked at random 
    with replacement
  '''
  
  '''
    max_depth = Maximum depth of decision trees in our forest
    criterion = criterion to be used for splitting nodes
    min_samples_split = No of min samples to be present to split a node
    min_samples_leaf = Minimum number of samples to be present in leaf node
    n_trees = No of trees in our forest
    max_features = maximum features to be considered at each node split
  '''

  def __init__(self, max_depth = None, criterion = "gini", min_samples_split = 2,
                   min_samples_leaf = 1, n_trees = 100, max_features = None):
    self.max_depth = max_depth
    self.criterion = criterion
    self.min_samples_split = min_samples_split
    self.min_samples_leaf = min_samples_leaf
    self.n_trees = n_trees
    self.decision_tree_list = None
    self.max_features = max_features
    self.label = None

  def fit(self, X_train, y_train):
    self.label = y_train.name
    decision_tree_list = []

    #we create the given number of trees for the forest 
    for i in range(self.n_trees):
      sample_array = []
      sample_class_array = []
      
      for i in range(X_train.shape[0]):
        '''
          we add data rows at random with replacement to our data.
          Note that the size of sample is same as population. But the sample 
          has data rows at random with replacement, so we can have 
          duplicate rows in sample.
        '''
        index = random.randint(0, X_train.shape[0]-1)
        sample_array.append(X_train.iloc[index])
        sample_class_array.append(y_train.iloc[index])
      sample_data = pd.DataFrame(sample_array, columns = headers[1:-1])
      sample_data_class = pd.DataFrame(sample_class_array, 
                                       columns = [y_train.name])
      decision_tree_classifier = DecisionTreeClassifier(
          max_depth = self.max_depth, criterion = self.criterion, 
          min_samples_split = self.min_samples_split, 
          min_samples_leaf = self.min_samples_leaf,
          max_features = self.max_features)
      decision_tree_classifier = decision_tree_classifier.fit(
          sample_data, sample_data_class)
      decision_tree_list.append(decision_tree_classifier)

    self.decision_tree_list = decision_tree_list
    return self

  '''
    Right now we have hard coded predict to just work on our breast-cancer-
    wisconsin data
  '''
  def predict(self, X_test):
    results = []
    for decision_tree in self.decision_tree_list:
      results.append(decision_tree.predict(X_test))
    final_result = []
    results = np.asarray(results).transpose(1,0)
    for result in results:
      count_2, count_4 = 0 , 0
      for data in result:
        if data == 2:
          count_2 += 1
        else :
          count_4 += 1
      final_result.append(2 if count_2 >= count_4
                          else 4)
    return pd.DataFrame(final_result, columns = [self.label])



2. What is boosting, bagging and  stacking?
Which class does random forests belong to and why?

Answer: 

These 3 techniques have one thing in common that they all in some way aggregate 
various ML models in a deterministic way to perform prediction or other tasks.
<br>
<br>
<u>1.Bagging</u> : Generate additional data for your model using repetetion i.e
random choices with replacement. This helps to decrease variance and helps in tuning our model. It is parallel in nature.(We are not implementing it parallely here)
<br>
<br>
<u>2.Boosting</u> : We create subsets of original data and aggregate various 
average performing models to create our final model. It is different from
bagging as the subsets are not at random, one subset depends on the subset used for previous models i.e it is sequential.
<br>
<br>
<u>3.Stacking</u> : It is similar to boosting but uses another model to estimate the input with the output.

<br>
Random forests belong to the class of bagging because we take a number of 
decision trees and then to each tree we give a sample which is created from 
originial data at random with replacement and then the results are aggregated
to give the final result.

3. Implement random forest algorithm using different decision trees . 

In [ ]:
'''
*****

Note: We have hardcoded our random forest model to work on the breast cancer data
that we are using.Also we have developed it only for binary classification. 

ToDo: Make the random forest implementation generic. For now we use hardcoded 
implementation

'''
model_1 = RandomForestBinaryClassifier(max_features = "sqrt",
                                     n_trees = 128)

model_1 = model_1.fit(X_train, y_train)

y_pred = model_1.predict(X_test)

print("Accuracy using random forest of decision trees using criterion gini is {:.3f}".
      format(metrics.accuracy_score(y_test, y_pred)))

model_2 = RandomForestBinaryClassifier(max_features = "sqrt",
                                     n_trees = 128, criterion = "entropy")

model_2 = model_2.fit(X_train, y_train)

y_pred = model_2.predict(X_test)

print("Accuracy using random forest of decision trees using criterion entropy is {:.3f}".
      format(metrics.accuracy_score(y_test, y_pred)))

Accuracy using random forest of decision trees using criterion gini is 0.964
Accuracy using random forest of decision trees using criterion entropy is 0.971


4. Report the accuracies obtained after using the Random forest algorithm and compare it with the best accuracies obtained with the decision trees. 

<br>
<b>1.Best accuracy we obtained in decision trees was 0.964. With Random forest we obtain an accuracy of 0.971 with criterion being entropy. Random forest performs better than decision tree
in this case which is as expected by us.
<br>
<br>

5. Submit your solution as a separate pdf in the final zip file of your submission


Compute a decision tree with the goal to predict the food review based on its smell, taste and portion size.

(a) Compute the entropy of each rule in the first stage.

(b) Show the final decision tree. Clearly draw it.

Submit a handwritten response. Clearly show all the steps.

